![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Proyecto 2 - Clasificación de género de películas

El propósito de este proyecto es que puedan poner en práctica, en sus respectivos grupos de trabajo, sus conocimientos sobre técnicas de preprocesamiento, modelos predictivos de NLP, y la disponibilización de modelos. Para su desarrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 2: Clasificación de género de películas"

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 8. Sin embargo, es importante que avancen en la semana 7 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 8, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/t/2c54d005f76747fe83f77fbf8b3ec232).

## Datos para la predicción de género en películas

![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/moviegenre.png)

En este proyecto se usará un conjunto de datos de géneros de películas. Cada observación contiene el título de una película, su año de lanzamiento, la sinopsis o plot de la película (resumen de la trama) y los géneros a los que pertenece (una película puede pertenercer a más de un género). Por ejemplo:
- Título: 'How to Be a Serial Killer'
- Plot: 'A serial killer decides to teach the secrets of his satisfying career to a video store clerk.'
- Generos: 'Comedy', 'Crime', 'Horror'

La idea es que usen estos datos para predecir la probabilidad de que una película pertenezca, dada la sinopsis, a cada uno de los géneros.

Agradecemos al profesor Fabio González, Ph.D. y a su alumno John Arevalo por proporcionar este conjunto de datos. Ver https://arxiv.org/abs/1702.01992

## Preprocesamiento de datos (10 puntos)

Los datos de entrenamiento se dividen en datos de entrenamiento y validación. (2 puntos)

Se preprocesan los datos de texto correctamente al ajustar sobre los datos de entrenamiento (.fit_transform()) y al transformar los datos del set de validación (.transform()). (8 puntos)

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación librerías
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\karen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
def lematzar(text):
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\W+', ' ', text) 
    text = text.lower() 
    tokens = text.split()
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stopwords.words('english')]
    return ' '.join(tokens)

In [5]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

In [6]:
# Visualización datos de entrenamiento
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [7]:
# Visualización datos de test
dataTesting.head()

,year,title,plot
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
4,1978,Midnight Express,"the true story of billy hayes , an american c..."
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


In [8]:
dataTraining['plot'] = dataTraining['plot'].apply(lematzar)
dataTesting['plot'] = dataTesting['plot'].apply(lematzar)

In [9]:
# Definición de variables predictoras (X)
tfidf = TfidfVectorizer(max_features=250000, ngram_range=(1, 2))
X_tfidf = tfidf.fit_transform(dataTraining['plot']).toarray()

In [10]:
# Definición de variable de interés (y)
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))
mlb = MultiLabelBinarizer()
y_genres = mlb.fit_transform(dataTraining['genres'])

In [11]:
# Separación de variables predictoras (X) y variable de interés (y) en set de entrenamiento y test usandola función train_test_split
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_tfidf, y_genres, test_size=0.33, random_state=42)

MemoryError: Unable to allocate 9.85 GiB for an array with shape (5289, 250000) and data type float64

In [67]:
# red neuronal
model = Sequential()

model.add(Dense(512, input_dim=X_train.shape[1], activation='sigmoid'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(y_train_genres.shape[1], activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit(X_train, y_train_genres, 
          epochs=50, 
          batch_size=16, 
          validation_split=0.2, 
          callbacks=[early_stopping])


y_pred_genres = model.predict(X_test)

print(f'Roc Auc: {roc_auc_score(y_test_genres, y_pred_genres, average="macro")}')

X_test_tfidf = tfidf.transform(dataTesting['plot']).toarray()

# Prediccion
y_pred_test_genres = model.predict(X_test_tfidf)


cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']



Epoch 1/50
265/265 [==============================] - 35s 128ms/step - loss: 0.4071 - accuracy: 0.2671 - val_loss: 0.2937 - val_accuracy: 0.2013
Epoch 2/50
265/265 [==============================] - 33s 124ms/step - loss: 0.1919 - accuracy: 0.4439 - val_loss: 0.2672 - val_accuracy: 0.3299
Epoch 3/50
265/265 [==============================] - 33s 125ms/step - loss: 0.0979 - accuracy: 0.5001 - val_loss: 0.2342 - val_accuracy: 0.3100
Epoch 4/50
265/265 [==============================] - 33s 125ms/step - loss: 0.0559 - accuracy: 0.4942 - val_loss: 0.2347 - val_accuracy: 0.3147
Epoch 5/50
265/265 [==============================] - 33s 125ms/step - loss: 0.0362 - accuracy: 0.4855 - val_loss: 0.2323 - val_accuracy: 0.3280
Epoch 6/50
265/265 [==============================] - 33s 124ms/step - loss: 0.0268 - accuracy: 0.4852 - val_loss: 0.2435 - val_accuracy: 0.3251
Epoch 7/50
265/265 [==============================] - 33s 124ms/step - loss: 0.0211 - accuracy: 0.4826 - val_loss: 0.2424 - val_ac

In [69]:
# Guardar predicciones en formato exigido en la competencia de kaggle
res = pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols)
res.to_csv('pred_genres_text_fin.csv', index_label='ID')
res.head()

,p_Action,p_Adventure,p_Animation,p_Biography,p_Comedy,p_Crime,p_Documentary,p_Drama,p_Family,p_Fantasy,...,p_Musical,p_Mystery,p_News,p_Romance,p_Sci-Fi,p_Short,p_Sport,p_Thriller,p_War,p_Western
1,0.033687,0.014942,0.004713,0.008058,0.432662,0.052281,0.005701,0.663601,0.007353,0.033854,...,0.016409,0.033562,0.000664,0.788764,0.010823,0.010431,0.004232,0.151632,0.003861,0.011224
4,0.117356,0.004386,0.002393,0.030819,0.174117,0.596621,0.011359,0.536384,0.002059,0.005508,...,0.005967,0.004575,0.000631,0.012130,0.002535,0.008733,0.005017,0.627806,0.007664,0.004538
5,0.034265,0.004695,0.002629,0.004464,0.078509,0.918770,0.009639,0.975472,0.001773,0.002573,...,0.001178,0.209272,0.000464,0.027168,0.004451,0.004488,0.002290,0.753639,0.007477,0.003961
6,0.023620,0.011949,0.007240,0.020782,0.070493,0.022431,0.017375,0.885331,0.013209,0.016161,...,0.007262,0.033313,0.001293,0.253076,0.012531,0.013073,0.005517,0.580874,0.033193,0.014144
7,0.005189,0.009967,0.018728,0.003754,0.065545,0.029593,0.006065,0.128938,0.034565,0.061670,...,0.002266,0.023921,0.000565,0.014612,0.164408,0.013698,0.002000,0.265330,0.001642,0.004846
